In [557]:
import pandas as pd 
import nltk
import re
from nltk.stem import PorterStemmer
import string
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [558]:
df = pd.read_csv('df_sanfranciso_clean.csv')

In [559]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      170 non-null    int64  
 1   guest_favorite  170 non-null    bool   
 2   rating          170 non-null    float64
 3   number_reviews  170 non-null    int64  
 4   type_host       170 non-null    object 
 5   hosting_time    170 non-null    int64  
 6   price           170 non-null    float64
 7   currency        170 non-null    object 
 8   all_reviews     170 non-null    object 
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 10.9+ KB


In [560]:
df.tail()

,Unnamed: 0,guest_favorite,rating,number_reviews,type_host,hosting_time,price,currency,all_reviews
165,165,True,5.00,12,Superhost,156,170.0,€,"['Eric and his wife, Flo, were warm and welcom..."
166,166,True,4.99,115,no_category,24,179.0,€,"[""Kevin and his family were excellent hosts. A..."
167,167,True,4.97,79,Superhost,96,136.0,€,['Audrey was super kind and amazing! She has l...
168,168,True,4.91,46,Superhost,144,49.0,€,['Jia was a wonderful guest. She goes out her ...
169,169,True,4.94,247,Superhost,96,210.0,€,"['Great stay with a beautiful, inspiring view...."


## Limpiamos el texto

In [561]:
df_nlp = pd.DataFrame(df[['Unnamed: 0', 'all_reviews']])

In [562]:
def process_reviews(reviews_str):
    try:
        # Eliminar los corchetes y comillas al inicio y al final
        reviews_str_cleaned = reviews_str.strip("[]'\"")
        
        # Separar las reseñas por comas (o cualquier otro delimitador)
        reviews_list = reviews_str_cleaned.split("', '")
        
        # Limpiar espacios en blanco en cada reseña
        processed_reviews = [review.strip() for review in reviews_list]
        
        return processed_reviews
    except:
        return []

# Aplicar la función de procesamiento a la columna 'all_reviews'
df_nlp['processed_reviews'] = df_nlp['all_reviews'].apply(process_reviews)

# Mostrar las primeras filas del resultado
df_nlp.head()

,Unnamed: 0,all_reviews,processed_reviews
0,0,['Highly recommend Christine’s place!! We had ...,[Highly recommend Christine’s place!! We had a...
1,1,['Angella is a friendly host who made us feel ...,[Angella is a friendly host who made us feel v...
2,2,['Great neighborhood. Walkable to parks and re...,[Great neighborhood. Walkable to parks and res...
3,3,"['Great place, great value, great host, great ...","[Great place, great value, great host, great l..."
4,4,['Very clean and very well-equipped accommodat...,[Very clean and very well-equipped accommodati...


In [563]:
df_nlp['processed_reviews'][0]

['Highly recommend Christine’s place!! We had a wonderful stay!! This space is beautiful, spacious, and very comfortable. It was very clean, well-appointed, and the location is so peaceful and serene! We really enjoyed the balcony too- great views of the city! Christine was a fantastic host— highly responsive and helpful. Hope we can stay again sometime!',
 'Had a lovely stay! Christine was very helpful and communicative with check in/check out instructions.',
 'As a former SF local, this was a fantastic place to spend a week. Parking was a snap, the view was gorgeous, and it was close to everything. Would absolutely stay again.',
 'Amazing and comfortable stay. Great location in Noe Valley. Hosts are very responsive and friendly. Would highly recommend!',
 'Everything was exactly as described. The view of the city was even better in person than online. Lots of restaurants and shops in walking distance. Also got to meet our hosts and say goodbye. Definitely recommend staying here :)',


In [564]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[-—]+', '', text)
    stopwords = nltk.corpus.stopwords.words("english")
    text = ' '.join([word for word in text.split() if word not in stopwords])
    #Eliminar espacios en blanco
    text = re.sub(r'\s+', ' ', text).strip()
    return text


Noe he usado stemming porque daba palabra sin sentido 

In [565]:
df_nlp['cleaned_reviews'] = df_nlp['processed_reviews'].apply(lambda reviews: [clean_text(review) for review in reviews])

In [566]:
df_nlp['cleaned_reviews'][0]

['highly recommend christine’s place wonderful stay space beautiful spacious comfortable clean wellappointed location peaceful serene really enjoyed balcony great views city christine fantastic host highly responsive helpful hope stay sometime',
 'lovely stay christine helpful communicative check incheck instructions',
 'former sf local fantastic place spend week parking snap view gorgeous close everything would absolutely stay',
 'amazing comfortable stay great location noe valley hosts responsive friendly would highly recommend',
 'everything exactly described view city even better person online lots restaurants shops walking distance also got meet hosts say goodbye definitely recommend staying',
 'christine wonderful helped coordinate stay mother communicative responsive mom found space comfortable clean like pictures also said view amazing positive experience would highly recommend',
 'place really nice quiet clean everything need would definitely book',
 'christine jonathan’s plac

Limpiar las reviewds
Nota todo esto tienes que hacerlo por fila!


In [567]:
#Crear una fila por reseña para pdoer calcular el bag of words y td-idf individual y asi despues hacer el promedio  
df_expanded = df_nlp.explode('cleaned_reviews').reset_index(drop=True)

In [568]:
df_expanded

,Unnamed: 0,all_reviews,processed_reviews,cleaned_reviews
0,0,['Highly recommend Christine’s place!! We had ...,[Highly recommend Christine’s place!! We had a...,highly recommend christine’s place wonderful s...
1,0,['Highly recommend Christine’s place!! We had ...,[Highly recommend Christine’s place!! We had a...,lovely stay christine helpful communicative ch...
2,0,['Highly recommend Christine’s place!! We had ...,[Highly recommend Christine’s place!! We had a...,former sf local fantastic place spend week par...
3,0,['Highly recommend Christine’s place!! We had ...,[Highly recommend Christine’s place!! We had a...,amazing comfortable stay great location noe va...
4,0,['Highly recommend Christine’s place!! We had ...,[Highly recommend Christine’s place!! We had a...,everything exactly described view city even be...
...,...,...,...,...
10085,169,"['Great stay with a beautiful, inspiring view....","[Great stay with a beautiful, inspiring view. ...",great stay sussan’s close 39 bus fisherman’s w...
10086,169,"['Great stay with a beautiful, inspiring view....","[Great stay with a beautiful, inspiring view. ...",great place fantastic views house clean felt s...
10087,169,"['Great stay with a beautiful, inspiring view....","[Great stay with a beautiful, inspiring view. ...",great place thank letting us stay really enjoy...
10088,169,"['Great stay with a beautiful, inspiring view....","[Great stay with a beautiful, inspiring view. ...",excellent location private beautiful view spac...


## Vectorizacion del texto y escalado

In [571]:

#obtener Bag-of-Words
vectorizer = CountVectorizer()
X_counts = vectorizer.fit_transform(df_expanded['cleaned_reviews'])

#escalar Bag-of-Words con TF-IDF
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

#convertir la matriz TF-IDF a array denso (si es necesario)
X_tfidf_dense = X_tfidf.toarray()

#agregar los vectores TF-IDF directamente como arrays NumPy en la columna del DataFrame
df_expanded['tfidf'] = list(X_tfidf_dense)  # Cada fila de 'tfidf' contiene un array NumPy

#agrupar por 'Unnamed: 0' y calcular la media de los vectores TF-IDF (usando NumPy directamente)
df_grouped = df_expanded.groupby('Unnamed: 0')['tfidf'].apply(lambda x: np.mean(np.vstack(x), axis=0))

#crear DataFrame con los resultados (sin convertir a lista)
df_result = pd.DataFrame({
    'Unnamed: 0': df_grouped.index,
    'mean_tfidf': df_grouped.values  # Esto se mantiene como arrays NumPy
})

# convertir la columna 'mean_tfidf' a una matriz
X= np.vstack(df_result['mean_tfidf'])
num_features = X.shape[1]
tfidf_column_names = [f'tfidf_{i}' for i in range(num_features)]

# Convertir la matriz en un dataframe para poder concatenarlo con el df_modelo
X_df = pd.DataFrame(X, columns=tfidf_column_names)



   Unnamed: 0                                         mean_tfidf
0           0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1           1  [0.0, 0.0, 0.008427974788195595, 0.0, 0.0, 0.0...
2           2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3           3  [0.0, 0.0, 0.0008721747921823991, 0.0, 0.0, 0....
4           4  [0.0, 0.0, 0.0, 0.007553277971936825, 0.0, 0.0...


In [572]:
X_df

,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,tfidf_6,tfidf_7,tfidf_8,tfidf_9,...,tfidf_14855,tfidf_14856,tfidf_14857,tfidf_14858,tfidf_14859,tfidf_14860,tfidf_14861,tfidf_14862,tfidf_14863,tfidf_14864
0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
1,0.000000,0.0,0.008428,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.002955,0.0,0.0
2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,0.000000,0.0,0.000872,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,0.000000,0.0,0.000000,0.007553,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
166,0.000000,0.0,0.001725,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
167,0.000000,0.0,0.000000,0.004931,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.009455,0.0,0.0,0.000000,0.0,0.0
168,0.000000,0.0,0.018801,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


In [573]:
print(df_expanded['cleaned_reviews'][0])

highly recommend christine’s place wonderful stay space beautiful spacious comfortable clean wellappointed location peaceful serene really enjoyed balcony great views city christine fantastic host highly responsive helpful hope stay sometime


### Análisis de sentimiento

In [574]:
from textblob import TextBlob

# Añadir columna de sentimiento (polarity de TextBlob, entre -1 y 1)
df_expanded['polarity'] = df_expanded['cleaned_reviews'].apply(lambda review: TextBlob(review).sentiment.polarity)
df_grouped_2 = df_expanded.groupby('Unnamed: 0')['polarity'].mean()
df_result_2 = pd.DataFrame({
    'Unnamed: 0': df_grouped_2.index,
    'mean_polarity': df_grouped_2.values
})
df_result_2= df_result_2.drop(columns= 'Unnamed: 0')


In [577]:
df_modelo = pd.concat([df, X_df, df_result_2], axis=1)

### Encoding de las variables categoricas

In [578]:
encoder = OneHotEncoder()
#Cogemos solo la primera columna dado que tenemos 2
guest_favourite = pd.DataFrame(encoder.fit_transform(df_modelo[['guest_favorite']]).toarray())[0]
type_host = pd.DataFrame(encoder.fit_transform(df_modelo[['type_host']]).toarray())

In [579]:
#encoding guest favorite
guest_favourite_encoded = encoder.fit_transform(df_modelo[['guest_favorite']]).toarray()
guest_favourite_columns = encoder.get_feature_names_out(['guest_favorite'])  # Obtener nombres de las columnas
guest_favourite_df = pd.DataFrame(guest_favourite_encoded, columns=guest_favourite_columns)

#encoding type_host
type_host_encoded = encoder.fit_transform(df_modelo[['type_host']]).toarray()
type_host_columns = encoder.get_feature_names_out(['type_host'])
type_host_df = pd.DataFrame(type_host_encoded, columns=type_host_columns)

### Escalado de las variables numéricas

In [597]:
scaler = MinMaxScaler()
df_modelo['number_reviews'] = scaler.fit_transform(df_modelo[['number_reviews']])
df_modelo['price'] = scaler.fit_transform(df_modelo[['price']])
df_modelo['hosting_time']= scaler.fit_transform(df_modelo[['hosting_time']])

In [581]:
df_modelo = df_modelo.drop(columns=['Unnamed: 0', 'guest_favorite', 'type_host', 'currency', 'all_reviews'])

In [582]:
df_modelo.head()

,rating,number_reviews,hosting_time,price,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,...,tfidf_14856,tfidf_14857,tfidf_14858,tfidf_14859,tfidf_14860,tfidf_14861,tfidf_14862,tfidf_14863,tfidf_14864,mean_polarity
0,4.94,0.068660,24,0.238516,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.461682
1,4.97,0.063123,12,0.130742,0.0,0.0,0.008428,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.002955,0.0,0.0,0.427599
2,4.92,0.146179,96,0.144876,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.419745
3,4.96,0.435216,108,0.042403,0.0,0.0,0.000872,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.410603
4,4.88,0.562569,108,0.024735,0.0,0.0,0.000000,0.007553,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.407585


In [583]:
df_modelo=pd.concat([df_modelo, guest_favourite_df, type_host_df], axis=1)

In [584]:
df_modelo.head()

,rating,number_reviews,hosting_time,price,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,...,tfidf_14860,tfidf_14861,tfidf_14862,tfidf_14863,tfidf_14864,mean_polarity,guest_favorite_True,type_host_Superhost,type_host_new_host,type_host_no_category
0,4.94,0.068660,24,0.238516,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.461682,1.0,1.0,0.0,0.0
1,4.97,0.063123,12,0.130742,0.0,0.0,0.008428,0.000000,0.0,0.0,...,0.0,0.0,0.002955,0.0,0.0,0.427599,1.0,1.0,0.0,0.0
2,4.92,0.146179,96,0.144876,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.419745,1.0,1.0,0.0,0.0
3,4.96,0.435216,108,0.042403,0.0,0.0,0.000872,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.410603,1.0,1.0,0.0,0.0
4,4.88,0.562569,108,0.024735,0.0,0.0,0.000000,0.007553,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.407585,1.0,1.0,0.0,0.0


### Modelo

In [600]:
X=df_modelo.drop(columns= ['rating', 'guest_favorite_True'])
y = df_modelo['rating']


def evaluate_models(X, y):
    #lista de modelos de regresión
    modelos = [
        LinearRegression(),
        KNeighborsRegressor(),
        DecisionTreeRegressor(),
        RandomForestRegressor(),
        SVR(),
        AdaBoostRegressor(),
        GradientBoostingRegressor()
    ]

    nombres = [
        'LinearRegression',
        'KNeighborsRegressor',
        'DecisionTreeRegressor',
        'RandomForestRegressor',
        'SVR',
        'AdaBoostRegressor',
        'GradientBoostingRegressor'
    ]

    #df para acumular los resultados
    resultados = pd.DataFrame(columns=['Model', 'R2', 'RMSE', 'MAE'])
    
    # Variables para guardar el mejor modelo
    mejor_r2 = -np.inf  # R2 más alto es mejor
    mejor_y_test = None
    mejor_y_pred = None
    mejor_modelo = None
    mejor_nombre = None

    #entrenar y evaluar cada modelo
    for modelo, nombre in zip(modelos, nombres):
        #división del dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    
        #entrenar el modelo
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        #métricas  evaluar modelo
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)

        # Guardar el mejor modelo según el R2
        if r2 > mejor_r2:
            mejor_r2 = r2
            mejor_y_test = y_test
            mejor_y_pred = y_pred
            mejor_modelo = modelo
            mejor_nombre = nombre

        # Crear un DataFrame con los resultados del modelo actual
        modelo_df = pd.DataFrame({
            'Model': [nombre],
            'R2': [r2],
            'RMSE': [rmse],
            'MAE': [mae]
        })

        # Mostrar las métricas
        print(f'Modelo: {nombre}\nR2: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}\n')

        # Agregar los resultados al DataFrame
        resultados = pd.concat([resultados, modelo_df], ignore_index=True)

    # Imprimir los valores de y_test y y_pred del mejor modelo
    resultados_comparados = np.column_stack((mejor_y_test, mejor_y_pred))
    print("\nValores reales vs predicciones del mejor modelo:")
    print(resultados_comparados)


    return resultados


In [601]:
resultados = evaluate_models(X=X, y=y)

Modelo: LinearRegression
R2: 0.3679, RMSE: 0.0323, MAE: 0.0253

Modelo: KNeighborsRegressor
R2: 0.3015, RMSE: 0.0339, MAE: 0.0286

Modelo: DecisionTreeRegressor
R2: -0.2370, RMSE: 0.0451, MAE: 0.0344



/var/folders/7h/qmh6bk5x34g5j30ygn0wjywr0000gn/T/ipykernel_4684/2636042241.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  resultados = pd.concat([resultados, modelo_df], ignore_index=True)


Modelo: RandomForestRegressor
R2: 0.3092, RMSE: 0.0337, MAE: 0.0289

Modelo: SVR
R2: -0.7814, RMSE: 0.0542, MAE: 0.0459

Modelo: AdaBoostRegressor
R2: 0.2270, RMSE: 0.0357, MAE: 0.0297

Modelo: GradientBoostingRegressor
R2: 0.2956, RMSE: 0.0341, MAE: 0.0296


Valores reales vs predicciones del mejor modelo:
[[4.98       4.97467729]
 [5.         4.97141191]
 [4.93       4.97721191]
 [4.99       4.95231426]
 [5.         5.        ]
 [4.93       4.98112267]
 [4.99       4.9593868 ]
 [4.98       4.92426069]
 [4.91       4.91671191]
 [4.98       4.98197349]
 [4.95       4.94848809]
 [5.         5.00908973]
 [4.95       4.94412524]
 [4.89       4.8888798 ]
 [5.         4.99293514]
 [4.89       4.93034254]
 [4.96       4.96950927]
 [4.93       4.98503938]
 [4.94       4.9214382 ]
 [4.87       4.93141852]
 [4.96       4.93098485]
 [4.93       4.96008943]
 [4.88       4.91963475]
 [4.91       4.95297184]
 [5.         5.00136369]
 [4.95       4.93488145]
 [4.98       4.98      ]
 [4.99       4.9